# Exploring Policies
In this notebook, I explore the 

### About the policies
ABAC policy file has extension .abac.
* userAttrib(uid, attribute1=value1, attribute2=value2, ...): is automatically assigned to an attribute named "uid".
* resourceAttrib(rid, attribute1=value1, attribute2=value2, ...): is automatically assigned to an attribute named "rid".
* rule(ruleName; uae; pae; ops; con)
* unremovableUserAttribs(attribute1, attribute2, ...)
* unremovableResourceAttribs(attribute1, attribute2, ...)

*value1, value2,... are atomic values or sets.*
* an ***atomic*** value is a string that starts with a character other than a left curly brace.
* a ***set*** has the form "{element1 element2 ...}".  note that elements of a set are separated by spaces, not commas.

***uae*** is a user attribute expression. it is a conjunction, with the conjuncts separated by commas.
* each conjunct has the form $a \in \{value1, value2, \dots \}$ where $a$ is a *single-valued* user attribute, or 
* $a \subseteq \{set1, set2, \dots \}$ (meaning that $a$ is a *superset* of one of the sets in *{set1 set2 ...}*) where $a$ is a *multi-valued* user attribute, and *set1, set2, ...,* are sets of the form *{value1+value2+...}*".
* note that sets (such as set1, set2, ...) that are nested inside another set are written with the elements separated by "+" (instead of " ") to simplify parsing.

***rae*** is a resource attribute expression.  the syntax is analogous to the syntax for user attribute expressions.

***ops*** is a set of operations.

***con*** is a constraint.
* it is a conjunction of atomic constraints, with the conjuncts separated by commas.
* an *atomic constraint* is a formula of the form "*aum > arm, aum ] ars*", or *aus=ars$*, where
    * **aus** is a single-valued user
    * **aum** is a multi-valued user attribute,
    * **ars** is a single-valued resource attribute, and
    * **arm** is a multi-valued resource attribute.
* note that $>$ denotes $\supseteq$ and "]" denotes $\ni$.

the sets of users, resources, operations, user attributes, permission attributes, **are not specified explicitly**.  They are implicitly defined as the sets containing the users, permissions, etc., that are mentioned in the policy.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Healthcare

In [139]:
# Load user and resource data
filepath = "data/HC-users.csv"
df_users = pd.read_csv(filepath) # Users

filepath = "data/HC-res.csv"
df_res = pd.read_csv(filepath)  # Resources

print(df_users.info(), df_res.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uname        23 non-null     object
 1   position     23 non-null     object
 2   uward        23 non-null     object
 3   specialties  23 non-null     object
 4   teams        23 non-null     object
 5   agentfor     23 non-null     object
dtypes: object(6)
memory usage: 1.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   rname        16 non-null     object
 1   type         16 non-null     object
 2   author       16 non-null     object
 3   patient      16 non-null     object
 4   topics       16 non-null     object
 5   tratingTeam  16 non-null     object
 6   rward        16 non-null     object
dtypes: object(7)
memory usage: 1.0+ KB
None None


In [140]:
len(df_users) * len(df_res)

368

In [141]:
result = 1
for col_name in df_res.columns[1:]:
    unique_values = df_res[col_name].value_counts().size
    print(col_name, unique_values)
    result = result * unique_values
print(result)

result = 1
for col_name in df_users.columns[1:]:
    unique_values = df_users[col_name].value_counts().size
    print(col_name, unique_values)
    result = result * unique_values
print(result)

type 2
author 13
patient 4
topics 5
tratingTeam 4
rward 2
4160
position 3
uward 3
specialties 6
teams 5
agentfor 3
810


In [142]:
# 0 = attribute = value
# 1 = attribute = attribute => attribute.value = attribute.value
# 2 = value in attribute => a in {a, b, c}
policy = [
    [
        [0, "position", "nurse"],
        [0, "type", "HR"],
        [1, "uward", "rward"]
    ],
    [
        [0, "type", "HR"],
        [1, "tratingTeam", "teams"]
    ],
    [
        [0, "type", "HR"],
        [1, "uname", "patient"]
    ],
    [
        [0, "type", "HR"],
        [1, "patient", "agentfor"]
    ],

    [
        [0, "type", "HRItem"],
        [1, "uname", "author"]
    ],
    [
        [0, "position", "nurse"],
        [0, "type", "HRItem"],
        [0, "topics", "nursing"],
        [1, "uward", "rward"]
    ],
    [
        [0, "type", "HRItem"],
        [1, "topics", "specialties"],
        [1, "tratingTeam", "teams"]
    ],
    [
        [0, "type", "HRItem"],
        [0, "topics", "note"],
        [1, "uname", "patient"]
    ],
    [
        [0, "type", "HRItem"],
        [0, "topics", "note"],
        [1, "patient", "agentfor"]
    ]
]

In [143]:
access_log = []
for user in df_users.values:
    for res in df_res.values:        
        access_log.append(list(user) + list(res))
        #print(list(user) + list(res))
access_log = pd.DataFrame(access_log)
access_log.columns = df_users.columns.append(df_res.columns)
access_log

,uname,position,uward,specialties,teams,agentfor,rname,type,author,patient,topics,tratingTeam,rward
0,oncNurse1,nurse,oncward,None,None,None,oncPat1oncItem,HRitem,oncDoc1,oncPat1,oncology,oncTeam1,oncward
1,oncNurse1,nurse,oncward,None,None,None,oncPat1nursingItem,HRitem,oncNurse2,oncPat1,nursing,oncTeam1,oncward
2,oncNurse1,nurse,oncward,None,None,None,oncPat1noteItem,HRitem,oncPat1,oncPat1,note,oncTeam1,oncward
3,oncNurse1,nurse,oncward,None,None,None,oncPat1HR,HR,none,oncPat1,none,oncTeam1,oncward
4,oncNurse1,nurse,oncward,None,None,None,oncPat2oncItem,HRitem,doc1,oncPat2,oncology,oncTeam2,oncward
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,carAgent2,None,None,None,None,carPat2,carPat1HR,HR,none,carPat1,none,carTeam1,carward
364,carAgent2,None,None,None,None,carPat2,carPat2carItem,HRitem,doc2,carPat2,cardiology,carTeam2,carward
365,carAgent2,None,None,None,None,carPat2,carPat2nursingItem,HRitem,carNurse2,carPat2,nursing,carTeam2,carward
366,carAgent2,None,None,None,None,carPat2,carPat2noteItem,HRitem,carAgent1,carPat2,note,carTeam2,carward


In [149]:
false_negs = []
positive_ar = []
for row in access_log.iloc:
    # Evaluation
    denies_count = 0    
    for rule in policy:
        res = True

        for item_rule in rule:    
            if item_rule[0] == 0:# Check type of typle => Attr=Value
                if row[item_rule[1]] != item_rule[2]:
                    res = False
                    break
            if item_rule[0] == 1: # => attr1.val = attr2.val
                if row[item_rule[1]] != row[item_rule[2]]:
                    res = False
                    break
            if item_rule[0] == 2: # val1 in {val1, val2, val3}
                if not row[item_rule[1]] in row[item_rule[2]]:
                    res = False
                    break
        if res == False:
            denies_count += 1
    if denies_count == len(policy):        
        false_negs.append(row)
    else:
        positive_ar.append(row)

false_negs = pd.DataFrame(false_negs)
positive_ar = pd.DataFrame(positive_ar)
print("FN;", len(false_negs), "\t\tTP:", len(positive_ar))


FN; 343 		TP: 25


In [145]:
false_negs = pd.DataFrame(false_negs)
false_negs

,uname,position,uward,specialties,teams,agentfor,rname,type,author,patient,topics,tratingTeam,rward
0,oncNurse1,nurse,oncward,None,None,None,oncPat1oncItem,HRitem,oncDoc1,oncPat1,oncology,oncTeam1,oncward
1,oncNurse1,nurse,oncward,None,None,None,oncPat1nursingItem,HRitem,oncNurse2,oncPat1,nursing,oncTeam1,oncward
2,oncNurse1,nurse,oncward,None,None,None,oncPat1noteItem,HRitem,oncPat1,oncPat1,note,oncTeam1,oncward
4,oncNurse1,nurse,oncward,None,None,None,oncPat2oncItem,HRitem,doc1,oncPat2,oncology,oncTeam2,oncward
5,oncNurse1,nurse,oncward,None,None,None,oncPat2nursingItem,HRitem,oncNurse1,oncPat2,nursing,oncTeam2,oncward
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,carAgent2,None,None,None,None,carPat2,carPat1noteItem,HRitem,carPat1,carPat1,note,carTeam1,carward
363,carAgent2,None,None,None,None,carPat2,carPat1HR,HR,none,carPat1,none,carTeam1,carward
364,carAgent2,None,None,None,None,carPat2,carPat2carItem,HRitem,doc2,carPat2,cardiology,carTeam2,carward
365,carAgent2,None,None,None,None,carPat2,carPat2nursingItem,HRitem,carNurse2,carPat2,nursing,carTeam2,carward


In [151]:
positive_ar["rname"].value_counts()

oncPat2HR    8
oncPat1HR    6
carPat2HR    6
carPat1HR    5
Name: rname, dtype: int64

In [147]:
a = false_negs[false_negs.type=="HR"]
a = a[a.teams!="None"]
a[["teams", "tratingTeam"]]

,teams,tratingTeam
71,oncTeam1,oncTeam2
75,oncTeam1,carTeam1
79,oncTeam1,carTeam2
83,oncTeam2,oncTeam1
91,oncTeam2,carTeam1
95,oncTeam2,carTeam2
103,oncTeam1,oncTeam2
107,oncTeam1,carTeam1
111,oncTeam1,carTeam2
115,oncTeam2,oncTeam1


In [52]:
class User:
    def __init__(self, name, position , uward , specialties , teams , agentfor) -> None:
        self.name = name
        self.position = position
        self.ward = uward
        self.specialties = specialties
        self.teams = teams
        self.agentfor = agentfor        

    def __str__(self) -> str:
        return self.name

class Users:
    def __init__(self, dataframe) -> None:
        self.df = dataframe
        self.attrs = dataframe.columns
        self.users = []        
    
    def build_list(self) -> None:
        """Create a list of user object"""
        rows = self.df.values        
        for i in rows:
            self.users.append(User(i[0], i[1], i[2], i[3], i[4], i[5]))
        print("Done")

    def users_size(self):
        return len(self.users)
    
class Rule:
    def __init__(self, attribute_expression) -> None:
        rule = []
        for item in attribute_expression: # For each attribute expression
            if item[0] == 0: # attribute = value

        pass